<a href="https://colab.research.google.com/github/Aneesh-Pal/Aneesh-PaL/blob/main/Aneesh_LR_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
pip install pyspark

In [96]:
from pyspark.sql import SparkSession
session=SparkSession.builder.appName("Regression_Gate").getOrCreate()

# Linear Regression

In [97]:
# importing dataset
data=session.read.csv("Gate_Admission3.csv",header=True,inferSchema=True)

In [98]:
data.show(5)

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+
|Serial No|GRE Score|TOTAL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|Gender|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|  Male|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|Female|
|        3|        0|        104|                3|3.0|3.5| 8.0|       1|           0.72|  Male|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|  Male|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|  Male|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+
only showing top 5 rows



In [99]:
data.columns

['Serial No',
 'GRE Score',
 'TOTAL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit',
 'Gender']

In [100]:
data.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOTAL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)
 |-- Gender: string (nullable = true)



In [101]:
#importing required functions 
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder

In [104]:
# Changing string datatypes of attributes using StringIndexer
str_idx=StringIndexer(inputCol='Gender',outputCol='new_Gender',handleInvalid='skip')

In [105]:
# assigning categorical variable into numerical using OnehotEncoder
onehot=OneHotEncoder(inputCol='new_Gender',outputCol='new_Gender2')

In [106]:
# assembling features using VectorAssembler
vec_assem=VectorAssembler(inputCols=['Serial No',
 'GRE Score',
 'TOTAL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research'],outputCol='allfeatures')

In [107]:
# importing model 
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol='allfeatures',labelCol='Chance of Admit')

In [110]:
from pyspark.ml import Pipeline
mypipe=Pipeline(stages=[str_idx,onehot,vec_assem,lr])

In [111]:
training,test=data.randomSplit([0.8,0.2])

In [112]:
training.show(5)

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+
|Serial No|GRE Score|TOTAL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|Gender|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|Female|
|        3|        0|        104|                3|3.0|3.5| 8.0|       1|           0.72|  Male|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|  Male|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|  Male|
|        7|      321|        109|                0|3.0|4.0| 8.2|       1|           0.75|Female|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+
only showing top 5 rows



In [113]:
fitmodel=mypipe.fit(training)

In [118]:
result = fitmodel.transform(test)
result.show(20)

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+----------+-------------+--------------------+------------------+
|Serial No|GRE Score|TOTAL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|Gender|new_Gender|  new_Gender2|         allfeatures|        prediction|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+------+----------+-------------+--------------------+------------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|  Male|       0.0|(1,[0],[1.0])|[1.0,337.0,118.0,...|0.9216101032626984|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|  Male|       0.0|(1,[0],[1.0])|[4.0,322.0,110.0,...|0.7219238209765597|
|        9|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|  Male|       0.0|(1,[0],[1.0])|[9.0,302.0,102.0,...|0.5442873176559213|
|       11|      325|       

In [115]:
from pyspark.ml.evaluation import RegressionEvaluator

In [119]:
eval_obj=RegressionEvaluator(predictionCol="prediction",labelCol="Chance of Admit",metricName="r2")

In [120]:
accuracy=eval_obj.evaluate(result)
print(accuracy)

0.8225638734917513
